In [1]:
# coding=utf-8
import re
import jieba
import random
from tensorflow.contrib import learn

class Data_Prepare(object):

    def readfile(self, filename):
        texta = []
        textb = []
        tag = []
        # 读取texta,textb,tag,并且分词
        with open(filename, 'r',encoding='utf-8') as f:
            for line in f.readlines():
                line = line.strip().split("\t")
                texta.append(self.pre_processing(line[0]))
                textb.append(self.pre_processing(line[1]))
                tag.append(line[2])
        # shuffle
        index = [x for x in range(len(texta))]
        random.shuffle(index)
        texta_new = [texta[x] for x in index]
        textb_new = [textb[x] for x in index]
        tag_new = [tag[x] for x in index]
        # 类别个数，构成[0,1]多分类向量
        type = list(set(tag_new))
        dicts = {}
        tags_vec = []
        for x in tag_new:
            if x not in dicts.keys():
                dicts[x] = 1
            else:
                dicts[x] += 1
            temp = [0] * len(type)
            temp[int(x)] = 1
            tags_vec.append(temp)
        print(dicts)
        return texta_new, textb_new, tags_vec

    def pre_processing(self, text):
        # 删除（）里的内容
        text = re.sub('（[^（.]*）', '', text)
        # 只保留中文部分
        text = ''.join([x for x in text if '\u4e00' <= x <= '\u9fa5'])
        # 利用jieba进行分词
        words = ' '.join(jieba.cut(text)).split(" ")
        # 不分词
        words = [x for x in ''.join(words)]
        return ' '.join(words)
        
    def build_vocab(self, sentences, path):
        print('start build_vocab...................')
        lens = [len(sentence.split(" ")) for sentence in sentences]
        max_length = max(lens)
        vocab_processor = learn.preprocessing.VocabularyProcessor(max_length)
        vocab_processor.fit(sentences)
        vocab_processor.save(path)

In [2]:
data_pre = Data_Prepare()
texta_new, textb_new, tags_vec = data_pre.readfile('data/1.txt')
print(texta_new[:10])


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\nie\AppData\Local\Temp\jieba.cache
Loading model cost 0.617 seconds.
Prefix dict has been built succesfully.


{'1': 79, '0': 45}
['如 何 做 生 意 赚 钱 啊', '天 天 酷 跑 怎 么 刷 积 分', '朋 友 过 生 日 唱 什 么 歌', '求 音 乐 世 界 不 是 我 们 的 家', '怎 么 查 通 话 记 录', '求 天 空 之 城 吉 他 谱 六 线 谱 简 单 点', '你 表 情 搞 笑 吗', '有 什 么 电 影 可 以 介 绍 吗', '百 度 上 的 财 富 值 有 什 么 用', '梦 见 抱 小 女 孩']


In [1]:
# coding=utf-8
import tensorflow as tf
from tensorflow.contrib import learn
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn import metrics
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3' 

import re
import jieba
import random
from tensorflow.contrib import learn

import tensorflow as tf
import math

class Data_Prepare(object):

    def readfile(self, filename):
        texta = []
        textb = []
        tag = []
        # 读取texta,textb,tag,并且分词
        with open(filename, 'r',encoding='utf-8') as f:
            for line in f.readlines():
                line = line.strip().split("\t")
                texta.append(self.pre_processing(line[0]))
                textb.append(self.pre_processing(line[1]))
                tag.append(line[2])
        # shuffle
        index = [x for x in range(len(texta))]
        random.shuffle(index)
        texta_new = [texta[x] for x in index]
        textb_new = [textb[x] for x in index]
        tag_new = [tag[x] for x in index]
        # 类别个数，构成[0,1]多分类向量
        type = list(set(tag_new))
        dicts = {}
        tags_vec = []
        for x in tag_new:
            if x not in dicts.keys():
                dicts[x] = 1
            else:
                dicts[x] += 1
            temp = [0] * len(type)
            temp[int(x)] = 1
            tags_vec.append(temp)
        print(dicts)
        return texta_new, textb_new, tags_vec

    def pre_processing(self, text):
        # 删除（）里的内容
        text = re.sub('（[^（.]*）', '', text)
        # 只保留中文部分
        text = ''.join([x for x in text if '\u4e00' <= x <= '\u9fa5'])
        # 利用jieba进行分词
        words = ' '.join(jieba.cut(text)).split(" ")
        # 不分词
        words = [x for x in ''.join(words)]
        return ' '.join(words)
        
    def build_vocab(self, sentences, path):
        print('start build_vocab...................')
        lens = [len(sentence.split(" ")) for sentence in sentences]
        max_length = max(lens)
        vocab_processor = learn.preprocessing.VocabularyProcessor(max_length)
        vocab_processor.fit(sentences)
        vocab_processor.save(path)

        
class Config(object):

    def __init__(self):
        self.embedding_size = 50  # 词向量维度
        self.hidden_num = 100  # 隐藏层规模
        self.l2_lambda = 0.01
        self.learning_rate = 0.001
        self.dropout_keep_prob = 0.5
        

        self.epoch = 10000
        self.Batch_Size = 256
        

class ESIM(object):

    def __init__(self, is_trainning, seq_length, class_num, vocabulary_size, embedding_size, hidden_num,
                 l2_lambda, learning_rate):
        self.is_trainning = is_trainning
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.hidden_num = hidden_num
        self.seq_length = seq_length
        self.class_num = class_num

        # init placeholder
        self.text_a = tf.placeholder(tf.int32, [None, seq_length], name='text_a')
        self.text_b = tf.placeholder(tf.int32, [None, seq_length], name='text_b')
        self.y = tf.placeholder(tf.int32, [None, class_num], name='y')
        print('text_a:',self.text_a.shape)
        
        # real length
        self.a_length = tf.placeholder(tf.int32, [None], name='a_length')
        self.b_length = tf.placeholder(tf.int32, [None], name='b_length')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # embedding层 论文中采用是预训练好的词向量 这里随机初始化一个词典 在训练过程中进行调整
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.vocab_matrix = tf.Variable(tf.truncated_normal(shape=[vocabulary_size, embedding_size],
                                                                stddev=1.0 / math.sqrt(embedding_size)),
                                            name='vacab_matrix')
            self.text_a_embed = tf.nn.embedding_lookup(self.vocab_matrix, self.text_a)
            self.text_b_embed = tf.nn.embedding_lookup(self.vocab_matrix, self.text_b)
            
            print('text_a_embed:',self.text_a_embed.shape)
            
        # Input Encoding
        with tf.name_scope('Input_Encoding'):
            a_bar = self.biLSTMBlock(self.text_a_embed, hidden_num, 'Input_Encoding/biLSTM', self.a_length)
            b_bar = self.biLSTMBlock(self.text_b_embed, hidden_num, 'Input_Encoding/biLSTM', self.b_length, isreuse=True)
            print('a_bar:',a_bar.shape)
        # Local Inference Modeling
        with tf.name_scope('Local_inference_Modeling'):
            # 计算a_bar与b_bar每个词语之间的相似度
            with tf.name_scope('word_similarity'):
                attention_weights = tf.matmul(a_bar, tf.transpose(b_bar, [0, 2, 1]))
                print('attention_weights:',attention_weights.shape)
                
                attentionsoft_a = tf.nn.softmax(attention_weights)
                attentionsoft_b = tf.nn.softmax(tf.transpose(attention_weights, [0, 2, 1]))
                #attentionsoft_b = tf.transpose(attentionsoft_b)
                a_hat = tf.matmul(attentionsoft_a, b_bar)
                b_hat = tf.matmul(attentionsoft_b, a_bar)
                print('a_hat:',a_hat.shape)

            # 计算m_a, m_b
            with tf.name_scope("compute_m_a/m_b"):
                a_diff = tf.subtract(a_bar, a_hat)
                a_mul = tf.multiply(a_bar, a_hat)

                b_diff = tf.subtract(b_bar, b_hat)
                b_mul = tf.multiply(b_bar, b_hat)

                # m_a = [a_bar, a_hat, a_bar - a_hat, a_bar 'dot' a_hat] (14)
                # m_b = [b_bar, b_hat, b_bar - b_hat, b_bar 'dot' b_hat] (15)
                self.m_a = tf.concat([a_bar, a_hat, a_diff, a_mul], axis=2)
                self.m_b = tf.concat([b_bar, b_hat, b_diff, b_mul], axis=2)
                
                print('m_a:',self.m_a.shape)

        with tf.name_scope("Inference_Composition"):
            v_a = self.biLSTMBlock(self.m_a, hidden_num, 'Inference_Composition/biLSTM', self.a_length)
            v_b = self.biLSTMBlock(self.m_b, hidden_num, 'Inference_Composition/biLSTM', self.b_length, isreuse=True)
            print('v_a:',v_a.shape)
            # average pool and max pool
            v_a_avg = tf.reduce_mean(v_a, axis=1)
            v_b_avg = tf.reduce_mean(v_b, axis=1)
            v_a_max = tf.reduce_max(v_a, axis=1)
            v_b_max = tf.reduce_max(v_b, axis=1)
            
            print('v_a_avg:',v_a_avg.shape)

            v = tf.concat([v_a_avg, v_a_max, v_b_avg, v_b_max], axis=1)
            print('v:',v.shape)
            
        with tf.name_scope("output"):
            initializer = tf.random_normal_initializer(0.0, 0.1)
            with tf.variable_scope('feed_foward_layer1'):
                inputs = tf.nn.dropout(v, self.dropout_keep_prob)
                outputs = tf.layers.dense(inputs, hidden_num, tf.nn.relu, kernel_initializer=initializer)
            with tf.variable_scope('feed_foward_layer2'):
                outputs = tf.nn.dropout(outputs, self.dropout_keep_prob)
                self.logits = tf.layers.dense(outputs, class_num, tf.nn.tanh, kernel_initializer=initializer)
            self.score = tf.nn.softmax(self.logits, name='score')
            self.prediction = tf.argmax(self.score, 1, name="prediction")

        with tf.name_scope('cost'):
            self.cost = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.y, logits=self.logits)
            self.cost = tf.reduce_mean(self.cost)
            # l2正则化
            weights = [v for v in tf.trainable_variables() if ('w' in v.name) or ('kernel' in v.name)]
            l2_loss = tf.add_n([tf.nn.l2_loss(w) for w in weights]) * l2_lambda
            self.loss = l2_loss + self.cost

        self.accuracy = tf.reduce_mean(
            tf.cast(tf.equal(tf.argmax(self.y, axis=1), self.prediction), tf.float32))

        if not is_trainning:
            return

        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), 5)

        optimizer = tf.train.AdamOptimizer(learning_rate)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))

    def biLSTMBlock(self, inputs, num_units, scope, seq_len=None, isreuse=False):
        with tf.variable_scope(scope, reuse=isreuse):
            cell_fw = tf.nn.rnn_cell.BasicLSTMCell(num_units)
            fw_lstm_cell = tf.contrib.rnn.DropoutWrapper(cell_fw, output_keep_prob=self.dropout_keep_prob)
            cell_bw = tf.nn.rnn_cell.BasicLSTMCell(num_units)
            bw_lstm_cell = tf.contrib.rnn.DropoutWrapper(cell_bw, output_keep_prob=self.dropout_keep_prob)

            (a_outputs, a_output_states) = tf.nn.bidirectional_dynamic_rnn(fw_lstm_cell, bw_lstm_cell,
                                                                           inputs,
                                                                           sequence_length=seq_len,
                                                                           dtype=tf.float32)
            a_bar = tf.concat(a_outputs, axis=2)
            return a_bar



        
con = Config()
data_pre = Data_Prepare()


class TrainModel(object):
    '''
        训练模型
        保存模型
    '''
    def pre_processing(self):
        print(' data_pre.readfile()......................')
        train_texta, train_textb, train_tag = data_pre.readfile('data/train.txt')
        data = []
        data.extend(train_texta)
        data.extend(train_textb)
        #训练集构造词典，并且保存
        data_pre.build_vocab(data, 'save_model/vocab/vocab.pickle')
        # 加载词典
        self.vocab_processor = learn.preprocessing.VocabularyProcessor.restore('save_model/vocab/vocab.pickle')
        # 构成id
        train_texta_embedding = np.array(list(self.vocab_processor.transform(train_texta)))
        train_textb_embedding = np.array(list(self.vocab_processor.transform(train_textb)))
        print('train_texta_embedding:',train_texta_embedding[:5])
        print('train_textb_embedding:',train_textb_embedding[:5])
        
        print(' data_pre.readfile()......................')
        dev_texta, dev_textb, dev_tag = data_pre.readfile('data/dev.txt')
        dev_texta_embedding = np.array(list(self.vocab_processor.transform(dev_texta)))
        dev_textb_embedding = np.array(list(self.vocab_processor.transform(dev_textb)))
        return train_texta_embedding, train_textb_embedding, np.array(train_tag), \
               dev_texta_embedding, dev_textb_embedding, np.array(dev_tag)

    def get_batches(self, texta, textb, tag, size):
        '''num_batch = int(len(texta) / con.Batch_Size)
        for i in range(num_batch):
            a = texta[i*con.Batch_Size:(i+1)*con.Batch_Size]
            b = textb[i*con.Batch_Size:(i+1)*con.Batch_Size]
            t = tag[i*con.Batch_Size:(i+1)*con.Batch_Size]
            yield a, b, t
        '''
        index = np.random.choice(len(texta),size,replace=True)
        a = texta[index]
        b = textb[index]
        t = tag[index]
        return a,b,t
        
    def get_length(self, trainX_batch):
        # sentence length
        lengths = []
        for sample in trainX_batch:
            count = 0
            for index in sample:
                if index != 0:
                    count += 1
                else:
                    break
            lengths.append(count)
        return lengths

    def trainModel(self):
        train_texta_embedding, train_textb_embedding, train_tag, \
        dev_texta_embedding, dev_textb_embedding, dev_tag = self.pre_processing()
        # 定义训练用的循环神经网络模型
        with tf.variable_scope('esim_model', reuse=None):
            # esim model ， 按照seq_length = 第一个的长度
            model = ESIM(True, seq_length=len(train_texta_embedding[0]),
                                    class_num=len(train_tag[0]),
                                    vocabulary_size=len(self.vocab_processor.vocabulary_),
                                    embedding_size=con.embedding_size,
                                    hidden_num=con.hidden_num,
                                    l2_lambda=con.l2_lambda,
                                    learning_rate=con.learning_rate)

        # 训练模型
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver()
            best_f1 = 0.0
            for time in range(con.epoch):
                
                model.is_trainning = True
                
                texta,textb,tag = self.get_batches(train_texta_embedding, train_textb_embedding, train_tag , con.Batch_Size)
                feed_dict = {
                    model.text_a: texta,
                    model.text_b: textb,
                    model.y: tag,
                    model.dropout_keep_prob: con.dropout_keep_prob,
                    model.a_length: np.array(self.get_length(texta)),
                    model.b_length: np.array(self.get_length(textb))
                }
                _, cost, accuracy = sess.run([model.train_op, model.loss, model.accuracy], feed_dict)
               
                if time%200==0:
                    print("training " + str(time + 1) + ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
                    
                    print("第" + str((time + 1)) + "次迭代的损失为：" + str(cost) + ";准确率为：" +str(accuracy))

                def dev_step():
                    """
                    Evaluates model on a dev set
                    """
                    texta,textb,tag = self.get_batches(dev_texta_embedding, dev_textb_embedding, dev_tag,size = 5000)
                    feed_dict = {
                        model.text_a: texta,
                        model.text_b: textb,
                        model.y: tag,
                        model.dropout_keep_prob: con.dropout_keep_prob,
                        model.a_length: np.array(self.get_length(texta)),
                        model.b_length: np.array(self.get_length(textb))
                    }
                    # 不训练
                    dev_cost, dev_accuracy, prediction = sess.run([model.loss, model.accuracy,
                                                                   model.prediction], feed_dict)
                   
                    print("验证集：loss {:g}, acc {:g}\n".format(dev_cost,dev_accuracy))
                    return dev_accuracy

                if time%500==0:
                    model.is_trainning = False
                    f1 = dev_step()

                    if f1 > best_f1:
                        best_f1 = f1
                        saver.save(sess, "save_model/ckpt/model.ckpt")
                        print("Saved model success\n")



train = TrainModel()
train.trainModel()

 data_pre.readfile()......................


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\nie\AppData\Local\Temp\jieba.cache
Loading model cost 0.972 seconds.
Prefix dict has been built succesfully.


{'0': 100192, '1': 138574}
start build_vocab...................
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
train_texta_embedding: [[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [15 16 17 18 19 20 21 22 23  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [24 25 26 27 28  6 29 30 31 32 33 34 15 35 36  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [10 37 38 39 40 41 42 43 44 45 46 47  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [48 49 50 51 52 53 54  5  6 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]]
train_textb_embedding: [[1731  326    5    6  608  935    9   10   11   12   13   14    0    0
     0    0    0    0    0    0    0    0    0    0   

In [1]:
# coding=utf-8
import tensorflow as tf
from tensorflow.contrib import learn
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn import metrics
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3' 

import re
import jieba
import random
from tensorflow.contrib import learn

import tensorflow as tf
import math

class Data_Prepare(object):

    def readfile(self, filename):
        texta = []
        textb = []
        tag = []
        # 读取texta,textb,tag,并且分词
        with open(filename, 'r',encoding='utf-8') as f:
            for line in f.readlines():
                line = line.strip().split("\t")
                texta.append(self.pre_processing(line[0]))
                textb.append(self.pre_processing(line[1]))
                tag.append(line[2])
        # shuffle
        index = [x for x in range(len(texta))]
        random.shuffle(index)
        texta_new = [texta[x] for x in index]
        textb_new = [textb[x] for x in index]
        tag_new = [tag[x] for x in index]
        # 类别个数，构成[0,1]多分类向量
        type = list(set(tag_new))
        dicts = {}
        tags_vec = []
        for x in tag_new:
            if x not in dicts.keys():
                dicts[x] = 1
            else:
                dicts[x] += 1
            temp = [0] * len(type)
            temp[int(x)] = 1
            tags_vec.append(temp)
        print(dicts)
        return texta_new, textb_new, tags_vec

    def pre_processing(self, text):
        # 删除（）里的内容
        text = re.sub('（[^（.]*）', '', text)
        # 只保留中文部分
        text = ''.join([x for x in text if '\u4e00' <= x <= '\u9fa5'])
        # 利用jieba进行分词
        words = ' '.join(jieba.cut(text)).split(" ")
        # 不分词
        words = [x for x in ''.join(words)]
        return ' '.join(words)
        
    def build_vocab(self, sentences, path):
        print('start build_vocab...................')
        lens = [len(sentence.split(" ")) for sentence in sentences]
        max_length = max(lens)
        vocab_processor = learn.preprocessing.VocabularyProcessor(max_length)
        vocab_processor.fit(sentences)
        vocab_processor.save(path)

        
class Config(object):

    def __init__(self):
        self.embedding_size = 50  # 词向量维度
        self.hidden_num = 100  # 隐藏层规模
        self.l2_lambda = 0.01
        self.learning_rate = 0.001
        self.dropout_keep_prob = 0.5
        

        self.epoch = 10
        self.Batch_Size = 50
        

class ESIM(object):

    def __init__(self, is_trainning, seq_length, class_num, vocabulary_size, embedding_size, hidden_num,
                 l2_lambda, learning_rate):
        self.is_trainning = is_trainning
        self.vocabulary_size = vocabulary_size
        self.embedding_size = embedding_size
        self.hidden_num = hidden_num
        self.seq_length = seq_length
        self.class_num = class_num

        # init placeholder
        self.text_a = tf.placeholder(tf.int32, [None, seq_length], name='text_a')
        self.text_b = tf.placeholder(tf.int32, [None, seq_length], name='text_b')
        self.y = tf.placeholder(tf.int32, [None, class_num], name='y')
        print('text_a:',self.text_a.shape)
        
        # real length
        self.a_length = tf.placeholder(tf.int32, [None], name='a_length')
        self.b_length = tf.placeholder(tf.int32, [None], name='b_length')
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # embedding层 论文中采用是预训练好的词向量 这里随机初始化一个词典 在训练过程中进行调整
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.vocab_matrix = tf.Variable(tf.truncated_normal(shape=[vocabulary_size, embedding_size],
                                                                stddev=1.0 / math.sqrt(embedding_size)),
                                            name='vacab_matrix')
            self.text_a_embed = tf.nn.embedding_lookup(self.vocab_matrix, self.text_a)
            self.text_b_embed = tf.nn.embedding_lookup(self.vocab_matrix, self.text_b)
            
            print('text_a_embed:',self.text_a_embed.shape)
            
        # Input Encoding
        with tf.name_scope('Input_Encoding'):
            a_bar = self.biLSTMBlock(self.text_a_embed, hidden_num, 'Input_Encoding/biLSTM', self.a_length)
            b_bar = self.biLSTMBlock(self.text_b_embed, hidden_num, 'Input_Encoding/biLSTM', self.b_length, isreuse=True)
            print('a_bar:',a_bar.shape)
        # Local Inference Modeling
        with tf.name_scope('Local_inference_Modeling'):
            # 计算a_bar与b_bar每个词语之间的相似度
            with tf.name_scope('word_similarity'):
                attention_weights = tf.matmul(a_bar, tf.transpose(b_bar, [0, 2, 1]))
                print('attention_weights:',attention_weights.shape)
                
                attentionsoft_a = tf.nn.softmax(attention_weights)
                attentionsoft_b = tf.nn.softmax(tf.transpose(attention_weights,[0, 2, 1]))
                #attentionsoft_b = tf.transpose(attentionsoft_b)
                a_hat = tf.matmul(attentionsoft_a, b_bar)
                b_hat = tf.matmul(attentionsoft_b, a_bar)
                print('a_hat:',a_hat.shape)

            # 计算m_a, m_b
            with tf.name_scope("compute_m_a/m_b"):
                a_diff = tf.subtract(a_bar, a_hat)
                a_mul = tf.multiply(a_bar, a_hat)

                b_diff = tf.subtract(b_bar, b_hat)
                b_mul = tf.multiply(b_bar, b_hat)

                # m_a = [a_bar, a_hat, a_bar - a_hat, a_bar 'dot' a_hat] (14)
                # m_b = [b_bar, b_hat, b_bar - b_hat, b_bar 'dot' b_hat] (15)
                self.m_a = tf.concat([a_bar, a_hat, a_diff, a_mul], axis=2)
                self.m_b = tf.concat([b_bar, b_hat, b_diff, b_mul], axis=2)
                
                print('m_a:',self.m_a.shape)

        with tf.name_scope("Inference_Composition"):
            v_a = self.biLSTMBlock(self.m_a, hidden_num, 'Inference_Composition/biLSTM', self.a_length)
            v_b = self.biLSTMBlock(self.m_b, hidden_num, 'Inference_Composition/biLSTM', self.b_length, isreuse=True)
            print('v_a:',v_a.shape)
            # average pool and max pool
            v_a_avg = tf.reduce_mean(v_a, axis=1)
            v_b_avg = tf.reduce_mean(v_b, axis=1)
            v_a_max = tf.reduce_max(v_a, axis=1)
            v_b_max = tf.reduce_max(v_b, axis=1)
            
            print('v_a_avg:',v_a_avg.shape)

            v = tf.concat([v_a_avg, v_a_max, v_b_avg, v_b_max], axis=1)
            print('v:',v.shape)
            
        with tf.name_scope("output"):
            initializer = tf.random_normal_initializer(0.0, 0.1)
            with tf.variable_scope('feed_foward_layer1'):
                inputs = tf.nn.dropout(v, self.dropout_keep_prob)
                outputs = tf.layers.dense(inputs, hidden_num, tf.nn.relu, kernel_initializer=initializer)
            with tf.variable_scope('feed_foward_layer2'):
                outputs = tf.nn.dropout(outputs, self.dropout_keep_prob)
                self.logits = tf.layers.dense(outputs, class_num, tf.nn.tanh, kernel_initializer=initializer)
            self.score = tf.nn.softmax(self.logits, name='score')
            self.prediction = tf.argmax(self.score, 1, name="prediction")

        with tf.name_scope('cost'):
            self.cost = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.y, logits=self.logits)
            self.cost = tf.reduce_mean(self.cost)
            # l2正则化
            weights = [v for v in tf.trainable_variables() if ('w' in v.name) or ('kernel' in v.name)]
            l2_loss = tf.add_n([tf.nn.l2_loss(w) for w in weights]) * l2_lambda
            self.loss = l2_loss + self.cost

        self.accuracy = tf.reduce_mean(
            tf.cast(tf.equal(tf.argmax(self.y, axis=1), self.prediction), tf.float32))

        if not is_trainning:
            return

        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, tvars), 5)

        optimizer = tf.train.AdamOptimizer(learning_rate)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))

    def biLSTMBlock(self, inputs, num_units, scope, seq_len=None, isreuse=False):
        with tf.variable_scope(scope, reuse=isreuse):
            cell_fw = tf.nn.rnn_cell.BasicLSTMCell(num_units)
            fw_lstm_cell = tf.contrib.rnn.DropoutWrapper(cell_fw, output_keep_prob=self.dropout_keep_prob)
            cell_bw = tf.nn.rnn_cell.BasicLSTMCell(num_units)
            bw_lstm_cell = tf.contrib.rnn.DropoutWrapper(cell_bw, output_keep_prob=self.dropout_keep_prob)

            (a_outputs, a_output_states) = tf.nn.bidirectional_dynamic_rnn(fw_lstm_cell, bw_lstm_cell,
                                                                           inputs,
                                                                           sequence_length=seq_len,
                                                                           dtype=tf.float32)
            a_bar = tf.concat(a_outputs, axis=2)
            return a_bar





# -*- coding: UTF-8 -*-
import tensorflow as tf
import data_prepare
from tensorflow.contrib import learn
import numpy as np
import esim_model
import config as config
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn import metrics
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3' # 屏蔽warning

con = Config()
data_pre = Data_Prepare()


class TrainModel(object):
    '''
        训练模型
        保存模型
    '''

    def pre_processing(self):

        train_texta, train_textb, train_tag = data_pre.readfile('data/train.txt')
        data = []
        data.extend(train_texta)
        data.extend(train_textb)
        # 训练集构造词典，并且保存
        data_pre.build_vocab(data, 'save_model/vocab/vocab.pickle')
        # 加载词典
        self.vocab_processor = learn.preprocessing.VocabularyProcessor.restore('save_model/vocab/vocab.pickle')
        # 中文句子，转化为id序列，固定长度
        train_texta_embedding = np.array(list(self.vocab_processor.transform(train_texta)))
        train_textb_embedding = np.array(list(self.vocab_processor.transform(train_textb)))
        print('train_texta_embedding:', train_texta_embedding[:5])
        print('train_textb_embedding:', train_textb_embedding[:5])


        dev_texta, dev_textb, dev_tag = data_pre.readfile('data/dev.txt')
        dev_texta_embedding = np.array(list(self.vocab_processor.transform(dev_texta)))
        dev_textb_embedding = np.array(list(self.vocab_processor.transform(dev_textb)))
        return train_texta_embedding, train_textb_embedding, np.array(train_tag), \
               dev_texta_embedding, dev_textb_embedding, np.array(dev_tag)
    # 1轮迭代进行batch循环，训练所有样本
    def get_batches(self, texta, textb, tag):
        num_batch = int(len(texta) / con.Batch_Size)
        for i in range(num_batch):
            a = texta[i*con.Batch_Size:(i+1)*con.Batch_Size]
            b = textb[i*con.Batch_Size:(i+1)*con.Batch_Size]
            t = tag[i*con.Batch_Size:(i+1)*con.Batch_Size]
            yield a, b, t
    # 获得句子的长度 = [句子1长度，句子2长度.....]
    def get_length(self, trainX_batch):
        # sentence length
        lengths = []
        for sample in trainX_batch:
            count = 0
            for index in sample:
                if index != 0:
                    count += 1
                else:
                    break
            lengths.append(count)
        return lengths

    def trainModel(self):
        train_texta_embedding, train_textb_embedding, train_tag, \
        dev_texta_embedding, dev_textb_embedding, dev_tag = self.pre_processing()
        # 定义训练用的循环神经网络模型
        with tf.variable_scope('esim_model', reuse=None):
            # esim model
            model = esim_model.ESIM(True, seq_length=len(train_texta_embedding[0]),
                                    class_num=len(train_tag[0]),
                                    vocabulary_size=len(self.vocab_processor.vocabulary_),
                                    embedding_size=con.embedding_size,
                                    hidden_num=con.hidden_num,
                                    l2_lambda=con.l2_lambda,
                                    learning_rate=con.learning_rate)

        # 训练模型
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver()
            best_f1 = 0.0
            for time in range(con.epoch):
                print("training " + str(time + 1) + ">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
                model.is_trainning = True
                loss_all = []
                accuracy_all = []
                # 1轮训练所有样本，分batch训练
                for texta, textb, tag in tqdm(
                        self.get_batches(train_texta_embedding, train_textb_embedding, train_tag)):
                    feed_dict = {
                        model.text_a: texta,
                        model.text_b: textb,
                        model.y: tag,
                        model.dropout_keep_prob: con.dropout_keep_prob,
                        model.a_length: np.array(self.get_length(texta)),
                        model.b_length: np.array(self.get_length(textb))
                    }
                    _, cost, accuracy = sess.run([model.train_op, model.loss, model.accuracy], feed_dict)
                    loss_all.append(cost)
                    accuracy_all.append(accuracy)

                print("第" + str((time + 1)) + "次迭代的损失为：" + str(np.mean(np.array(loss_all))) + ";准确率为：" +
                      str(np.mean(np.array(accuracy_all))))

                def dev_step():
                    """
                    Evaluates model on a dev set
                    """
                    loss_all = []
                    accuracy_all = []
                    predictions = []
                    for texta, textb, tag in tqdm(
                            self.get_batches(dev_texta_embedding, dev_textb_embedding, dev_tag)):
                        feed_dict = {
                            model.text_a: texta,
                            model.text_b: textb,
                            model.y: tag,
                            model.dropout_keep_prob: 1.0, # 测试
                            model.a_length: np.array(self.get_length(texta)),
                            model.b_length: np.array(self.get_length(textb))
                        }
                        dev_cost, dev_accuracy, prediction = sess.run([model.loss, model.accuracy,
                                                                       model.prediction], feed_dict)
                        loss_all.append(dev_cost)
                        accuracy_all.append(dev_accuracy)
                        predictions.extend(prediction)
                    y_true = [np.nonzero(x)[0][0] for x in dev_tag]
                    y_true = y_true[0:len(loss_all)*con.Batch_Size]
                    f1 = f1_score(np.array(y_true), np.array(predictions), average='weighted')
                    print('分类报告:\n', metrics.classification_report(np.array(y_true), predictions))
                    print("验证集：loss {:g}, acc {:g}, f1 {:g}\n".format(np.mean(np.array(loss_all)),
                                                                      np.mean(np.array(accuracy_all)), f1))
                    return f1

                model.is_trainning = False
                f1 = dev_step()

                if f1 > best_f1:
                    best_f1 = f1
                    saver.save(sess, "save_model/ckpt/model.ckpt")
                    print("Saved model success\n")



train = TrainModel()
train.trainModel()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\nie\AppData\Local\Temp\jieba.cache
Loading model cost 0.627 seconds.
Prefix dict has been built succesfully.


{'1': 138574, '0': 100192}
start build_vocab...................
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
train_texta_embedding: [[ 1  2  3  4  5  6  7  8  9 10 11  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [12 13 14 15 16 17 18  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [19 20 16 17 21 10 22  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [ 5  6  7  8  9 23 24 25 26 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]
 [27 28 29 30 31 32 33 34 35 36 37  9 38 39  4 40 17  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0]]
train_textb_embedding: [[   5    6    7    8    1    2    3   10    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0   

4775it [28:26,  2.59it/s]


第1次迭代的损失为：0.72860974;准确率为：0.6861822


176it [00:22,  8.75it/s]


...

分类报告:
               precision    recall  f1-score   support

           0       0.70      0.49      0.57      4398
           1       0.61      0.79      0.69      4402

   micro avg       0.64      0.64      0.64      8800
   macro avg       0.65      0.64      0.63      8800
weighted avg       0.65      0.64      0.63      8800

验证集：loss 0.750597, acc 0.638182, f1 0.629645

Saved model success

training 10>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


4775it [18:20,  4.62it/s]


第10次迭代的损失为：0.549713;准确率为：0.8074681


176it [00:10, 16.36it/s]


分类报告:
               precision    recall  f1-score   support

           0       0.69      0.45      0.55      4398
           1       0.59      0.79      0.68      4402

   micro avg       0.62      0.62      0.62      8800
   macro avg       0.64      0.62      0.61      8800
weighted avg       0.64      0.62      0.61      8800

验证集：loss 0.768634, acc 0.623068, f1 0.611865



In [2]:
# -*- coding: UTF-8 -*-
import tensorflow.contrib.learn as learn
import tensorflow as tf
import numpy as np
import data_prepare

data_pre = Data_Prepare()


class Infer(object):
    """
        ues model to predict classification.
    """
    def __init__(self):
        self.vocab_processor = learn.preprocessing.VocabularyProcessor.restore('save_model/vocab/vocab.pickle')
        self.checkpoint_file = tf.train.latest_checkpoint('save_model/ckpt')
        graph = tf.Graph()
        with graph.as_default():
            session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
            self.sess = tf.Session(config=session_conf)
            with self.sess.as_default():
                # Load the saved meta graph and restore variables
                saver = tf.train.import_meta_graph("{}.meta".format(self.checkpoint_file))
                saver.restore(self.sess, self.checkpoint_file)

                # Get the placeholders from the graph by name
                self.text_a = graph.get_operation_by_name("esim_model/text_a").outputs[0]
                self.text_b = graph.get_operation_by_name("esim_model/text_b").outputs[0]
                self.a_length = graph.get_operation_by_name("esim_model/a_length").outputs[0]
                self.b_length = graph.get_operation_by_name("esim_model/b_length").outputs[0]
                self.drop_keep_prob = graph.get_operation_by_name("esim_model/dropout_keep_prob").outputs[0]

                # Tensors we want to evaluate
                self.prediction = graph.get_operation_by_name("esim_model/output/prediction").outputs[0]
                self.score = graph.get_operation_by_name("esim_model/output/score").outputs[0]

    def infer(self, sentenceA, sentenceB):
        # transfer to vector
        sentenceA = [data_pre.pre_processing(sentenceA)]
        sentenceB = [data_pre.pre_processing(sentenceB)]
        vector_A = np.array(list(self.vocab_processor.transform(sentenceA)))
        vector_B = np.array(list(self.vocab_processor.transform(sentenceB)))
        feed_dict = {
            self.text_a: vector_A,
            self.text_b: vector_B,
            self.drop_keep_prob: 1.0,
            self.a_length: np.array([len(sentenceA[0].split(" "))]),
            self.b_length: np.array([len(sentenceB[0].split(" "))])
        }
        y, s = self.sess.run([self.prediction, self.score], feed_dict)
        return y, s


if __name__ == '__main__':
    infer = Infer()
    sentencea = '你点击详情'
    sentenceb = '您点击详情'
    print(infer.infer(sentencea, sentenceb))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from save_model/ckpt\model.ckpt
(array([1], dtype=int64), array([[0.40184772, 0.5981523 ]], dtype=float32))


In [4]:
import tensorflow as tf
import numpy as np
a = np.array([[
        [1,2,3],
        [4,5,6],
        [7,8,9]],
        [[11,22,33],
        [44,55,66],
        [77,88,99]]
        ])
print(a.shape)
result = tf.transpose(a, perm=[0, 2, 1])
with tf.Session() as sess:
    res = sess.run(result)
    print(res.shape)  
    print(res)

(2, 3, 3)
(2, 3, 3)
[[[ 1  4  7]
  [ 2  5  8]
  [ 3  6  9]]

 [[11 44 77]
  [22 55 88]
  [33 66 99]]]


In [5]:
import tensorflow as tf
import numpy as np
a = np.array([[
        [1,2,3],
        [4,5,6],
        [7,8,9]],
        [[11,22,33],
        [44,55,66],
        [77,88,99]]
        ])
print(a.shape)
result = tf.transpose(a)
with tf.Session() as sess:
    res = sess.run(result)
    print(res.shape)  
    print(res)

(2, 3, 3)
(3, 3, 2)
[[[ 1 11]
  [ 4 44]
  [ 7 77]]

 [[ 2 22]
  [ 5 55]
  [ 8 88]]

 [[ 3 33]
  [ 6 66]
  [ 9 99]]]


In [6]:
import tensorflow as tf
import numpy as np
a = np.array([[
        [1,2,3],
        [4,5,6],
        [7,8,9]],
        [[11,22,33],
        [44,55,66],
        [77,88,99]]
        ])
print(a.shape)
result = tf.transpose(a)
result = tf.transpose(result)
with tf.Session() as sess:
    res = sess.run(result)
    print(res.shape)  
    print(res)

(2, 3, 3)
(2, 3, 3)
[[[ 1  2  3]
  [ 4  5  6]
  [ 7  8  9]]

 [[11 22 33]
  [44 55 66]
  [77 88 99]]]
